In [1]:
# Inelegant but we are only loading three files:
topics_35 = open('../output/nmf_35_topics.txt', 'r').read().splitlines()
topics_45 = open('../output/nmf_45_topics.txt', 'r').read().splitlines()
topics_55 = open('../output/nmf_55_topics.txt', 'r').read().splitlines()

In [2]:
# The Jaccard coefficient is going to be our measure of similarity. 
# (This is entirely provisional, but it performs better here than the
# edit distance.)
def jaccard_similarity(query, document):
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

In [3]:
print("Jc = Jaccard coefficient")
print("========================")
for ik, iv in enumerate(topics_45):
    for jk, jv in enumerate(topics_35):
        if jaccard_similarity(iv.split(" "), jv.split(" ")) > 0.5:
            print(f"35-{ik} and 45-{jk} have a Jc of {jaccard_similarity(iv,jv):.2f}.")

Jc = Jaccard coefficient
35-0 and 45-0 have a Jc of 0.95.
35-1 and 45-1 have a Jc of 0.95.
35-2 and 45-2 have a Jc of 0.90.
35-3 and 45-3 have a Jc of 0.90.
35-4 and 45-4 have a Jc of 1.00.
35-5 and 45-5 have a Jc of 0.95.
35-7 and 45-7 have a Jc of 1.00.
35-8 and 45-8 have a Jc of 0.90.
35-9 and 45-9 have a Jc of 0.91.
35-10 and 45-10 have a Jc of 1.00.
35-11 and 45-11 have a Jc of 0.95.
35-13 and 45-13 have a Jc of 0.96.
35-14 and 45-14 have a Jc of 0.96.
35-16 and 45-16 have a Jc of 1.00.
35-17 and 45-17 have a Jc of 0.90.
35-18 and 45-18 have a Jc of 1.00.
35-19 and 45-22 have a Jc of 0.96.
35-20 and 45-20 have a Jc of 0.90.
35-21 and 45-21 have a Jc of 0.95.
35-23 and 45-23 have a Jc of 1.00.
35-25 and 45-25 have a Jc of 0.81.
35-27 and 45-31 have a Jc of 1.00.
35-28 and 45-28 have a Jc of 0.95.
35-29 and 45-29 have a Jc of 1.00.
35-32 and 45-24 have a Jc of 1.00.
35-35 and 45-34 have a Jc of 0.95.
35-36 and 45-30 have a Jc of 1.00.
35-40 and 45-32 have a Jc of 1.00.
